# 🧠 BNNR — Full Classification Demo

**Bulletproof Neural Network Recipe** — Automated Augmentation Search for PyTorch.

This notebook demonstrates the **complete BNNR classification pipeline** on **STL-10** (96×96 colour images, 10 classes). You'll see:

- All 8 built-in BNNR augmentations (7 unique + BasicAug) + **ICD/AICD** (XAI-driven, unique to BNNR)
- Torchvision wrapper augmentations alongside BNNR's own
- Automatic augmentation selection with branching decisions
- OptiCAM saliency maps (XAI) with per-class insights
- **Live dashboard** for real-time training monitoring

**Why STL-10?** The 96×96 resolution (vs MNIST 28×28) makes augmentation effects and XAI heatmaps clearly visible. 10 classes of natural images provide a non-trivial classification problem.

⏱ **Runtime:** ~15 min on Colab GPU (T4), ~40 min on CPU.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mateuszwalo/BNNR/blob/main/examples/classification/bnnr_classification_demo.ipynb)

## 1. Installation

Install BNNR with dashboard and visualization support. This takes ~30 seconds.

In [ ]:
%pip install -q "bnnr[dashboard]" matplotlib

## 2. Imports

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets, transforms

from bnnr import (
    BNNRConfig,
    BNNRTrainer,
    SimpleTorchAdapter,
    # Built-in augmentations (unique to BNNR)
    ChurchNoise,
    BasicAugmentation,
    DifPresets,
    Drust,
    LuxferGlass,
    ProCAM,
    Smugs,
    TeaStains,
    # External library wrappers
    TorchvisionAugmentation,
)

# ICD / AICD — BNNR's flagship XAI-driven augmentations
from bnnr.icd import ICD, AICD
from bnnr.xai_cache import XAICache

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
print(f"Using device: {DEVICE}")

## 3. Dataset — STL-10

STL-10 contains 5,000 training and 8,000 test images across 10 classes:
*airplane, bird, car, cat, deer, dog, horse, monkey, ship, truck.*

**Important:** Do **NOT** use `transforms.Normalize()`. BNNR augmentations operate on raw `[0, 1]` tensors — normalization destroys pixel values before augmentations are applied. Use `BatchNorm` layers inside your model instead.

The `IndexedDataset` wrapper adds a sample index to each item — this enables BNNR's XAI caching for ICD/AICD.

In [ ]:
STL10_CLASSES = [
    "airplane", "bird", "car", "cat", "deer",
    "dog", "horse", "monkey", "ship", "truck",
]


class IndexedDataset(Dataset):
    """Wraps (image, label) -> (image, label, index) for ICD/AICD cache."""
    def __init__(self, ds):
        self.ds = ds
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        img, lbl = self.ds[idx]
        return img, lbl, idx


# Raw ToTensor only — no Normalize!
transform = transforms.Compose([
    transforms.Resize((48, 48)),  # Smaller for faster Colab demo
    transforms.ToTensor(),
])

train_ds = datasets.STL10("data", split="train", download=True, transform=transform)
val_ds = datasets.STL10("data", split="test", download=True, transform=transform)

# Use a subset for faster demo (remove these for full training)
train_ds = Subset(train_ds, range(2000))
val_ds = Subset(val_ds, range(1000))

train_loader = DataLoader(IndexedDataset(train_ds), batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(IndexedDataset(val_ds), batch_size=64, num_workers=2)

print(f"Train: {len(train_ds):,}  Val: {len(val_ds):,}  Classes: {len(STL10_CLASSES)}")

## 4. Model

A compact VGG-style CNN (~150K params). `AdaptiveAvgPool2d` makes it resolution-agnostic.

The `target_layer` property tells BNNR which conv layer to use for:
- **OptiCAM** saliency maps (XAI visualization)
- **ICD/AICD** importance computation

In [ ]:
class DemoNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),  # OptiCAM target layer
            nn.BatchNorm2d(128), nn.ReLU(True),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(), nn.Dropout(0.3), nn.Linear(128, num_classes),
        )

    def forward(self, x):
        return self.classifier(self.features(x))

    @property
    def target_layer(self):
        return self.features[8]  # Last Conv2d


model = DemoNet(num_classes=10)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 5. Adapter & Config

`SimpleTorchAdapter` wraps your model for BNNR compatibility.
`BNNRConfig` controls the search strategy.

In [ ]:
adapter = SimpleTorchAdapter(
    model=model,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.05),
    optimizer=torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4),
    target_layers=[model.target_layer],
    device=DEVICE,
)

config = BNNRConfig(
    m_epochs=5,              # Train each candidate for 5 epochs
    max_iterations=4,        # 4 rounds of augmentation selection
    selection_metric="accuracy",
    selection_mode="max",
    device=DEVICE,
    xai_enabled=True,
    xai_method="opticam",
    report_dir="reports/stl10_demo",
    report_preview_size=512,
    report_xai_size=512,
    event_log_enabled=True,
    save_checkpoints=True,
    candidate_pruning_enabled=True,
    candidate_pruning_warmup_epochs=2,
    candidate_pruning_relative_threshold=0.5,
    seed=SEED,
)

print(f"Selection metric: {config.selection_metric}")
print(f"Metrics tracked:  {config.metrics}")
print(f"XAI method:       {config.xai_method}")
print(f"Pruning:          enabled (threshold=0.5)")

## 6. Augmentations — BNNR's Full Arsenal

We combine **all BNNR augmentation types** into one candidate pool:

| Category | Augmentations | What's Special |
|:---|:---|:---|
| **Built-in** | ChurchNoise, BasicAug, DifPresets, Drust, LuxferGlass, ProCAM, Smugs, TeaStains | Unique to BNNR — not available in any other library |
| **XAI-driven** | **ICD**, **AICD** | Use saliency maps to intelligently mask image regions |
| **Wrapper** | Torchvision ColorJitter, HFlip | Standard augmentations wrapped for BNNR |

BNNR will test each candidate and **automatically keep only those that improve accuracy**.

In [ ]:
# ── Precompute XAI cache for ICD/AICD ────────────────────────────────
print("Precomputing XAI saliency cache for ICD/AICD...")
xai_cache = XAICache(cache_dir="reports/stl10_demo/xai_cache")
written = xai_cache.precompute_cache(
    model=model,
    train_loader=train_loader,
    target_layers=[model.target_layer],
    n_samples=len(train_ds),
    method="opticam",
)
print(f"Cached {written} saliency maps")

# ── Build augmentation candidates ───────────────────────────────
augmentations = [
    # 8 built-in BNNR augmentations (7 unique + BasicAug for standard transforms)
    ChurchNoise(probability=0.5, intensity=0.5, random_state=SEED),
    BasicAugmentation(probability=0.5, intensity=0.5, random_state=SEED + 1),
    DifPresets(probability=0.5, intensity=0.7, random_state=SEED + 2),
    Drust(probability=0.5, intensity=0.5, random_state=SEED + 3),
    LuxferGlass(probability=0.5, intensity=0.5, random_state=SEED + 4),
    ProCAM(probability=0.5, random_state=SEED + 5),
    Smugs(probability=0.5, intensity=1.5, random_state=SEED + 6),
    TeaStains(probability=0.5, intensity=0.5, random_state=SEED + 7),

    # Torchvision wrappers (standard augmentations)
    TorchvisionAugmentation(
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
        name_override="tv_color_jitter",
        probability=0.5,
        random_state=SEED + 20,
    ),
    TorchvisionAugmentation(
        transforms.RandomHorizontalFlip(p=1.0),
        name_override="tv_hflip",
        probability=0.5,
        random_state=SEED + 21,
    ),
]

# ICD & AICD — XAI-driven augmentations (BNNR's flagship innovation)
icd = ICD(
    model=model, target_layers=[model.target_layer],
    threshold_percentile=70.0, explainer="opticam",
    use_cuda=(DEVICE != "cpu"), cache=xai_cache,
    tile_size=8, fill_strategy="gaussian_blur",
    probability=0.5, random_state=SEED + 50,
)
icd.name = "icd"
augmentations.append(icd)

aicd = AICD(
    model=model, target_layers=[model.target_layer],
    threshold_percentile=70.0, explainer="opticam",
    use_cuda=(DEVICE != "cpu"), cache=xai_cache,
    tile_size=8, fill_strategy="gaussian_blur",
    probability=0.5, random_state=SEED + 51,
)
aicd.name = "aicd"
augmentations.append(aicd)

print(f"\n{len(augmentations)} augmentation candidates:")
for i, aug in enumerate(augmentations, 1):
    tag = ""
    if aug.name in ("icd", "aicd"):
        tag = " [XAI-driven]"
    elif aug.name.startswith("tv_"):
        tag = " [torchvision]"
    else:
        tag = " [BNNR built-in]"
    print(f"  {i:2d}. {aug.name:<24s}  p={aug.probability:.2f}{tag}")

## 7. Dashboard

BNNR includes a **live dashboard** that should run alongside training. Use this section before `trainer.run()` so you can track progress from the first epoch.

What you should see:
- branch decision tree,
- KPI timeline (accuracy/loss/F1),
- per-class metrics,
- augmentation previews and XAI panels.

Tracking modes:
- **Local machine**: open the local URL printed by `start_dashboard(...)`; QR code is available in terminal workflows (`bnnr train --with-dashboard`).
- **Google Colab**: dashboard is embedded as an iframe.
- **Mobile from Colab (optional)**: set `NGROK_AUTHTOKEN` and use the generated public URL.



In [ ]:
# Start dashboard server — runs in a background thread, doesn't block training.

import os
import time as _time

try:
    from bnnr import start_dashboard
    dashboard_url = start_dashboard(
        run_root=config.report_dir,
        port=8080,
        auto_open=False,  # We handle display below
        build_frontend=False,  # Skip npm build on Colab
    )
    print(f"Dashboard server: {dashboard_url}")

    # On Google Colab: embed dashboard in notebook
    try:
        import google.colab  # noqa: F401
        from google.colab import output as colab_output

        _time.sleep(2)
        print("\n📊 Colab detected — embedding live dashboard below:")
        colab_output.serve_kernel_port_as_iframe(8080, height=900)

        # Optional: expose dashboard externally (phone/outside Colab session)
        # Requires: NGROK_AUTHTOKEN set in environment
        try:
            from pyngrok import ngrok

            token = os.environ.get("NGROK_AUTHTOKEN")
            if token:
                ngrok.set_auth_token(token)
                public_url = ngrok.connect(8080, "http").public_url
                print(f"🌍 Public dashboard URL: {public_url}")
            else:
                print("ℹ Optional public URL: install pyngrok and set NGROK_AUTHTOKEN.")
        except Exception as tunnel_exc:
            print(f"ℹ Tunnel not started: {tunnel_exc}")

    except ImportError:
        # Not on Colab — open URL manually
        print("Open the URL above in your browser to view the dashboard.")

except Exception as e:
    print(f"Dashboard not available: {e}")
    print("Training will continue without live monitoring.")
    print("Results are saved to the reports/ folder.")


## 8. Train with BNNR

This runs the full pipeline:
1. **Baseline** — train without augmentations to establish a reference
2. **Iteration 1** — test each candidate augmentation for `m_epochs=5`; select the best
3. **Iteration 2–4** — stack selected augmentations; test new candidates on top

At each step, BNNR generates:
- Accuracy / F1 / loss metrics
- Per-class accuracy breakdown
- OptiCAM saliency maps (XAI)
- Human-readable XAI insights and quality scores

**Candidate pruning** is enabled: candidates that fall behind by >50% after 2 epochs are stopped early, saving time.

In [ ]:
trainer = BNNRTrainer(adapter, train_loader, val_loader, augmentations, config)
result = trainer.run()

## 9. Results

In [ ]:
print("=" * 60)
print("  BNNR Classification Results")
print("=" * 60)
print(f"  Best augmentation path : {result.best_path}")
print(f"  Best metrics           : {result.best_metrics}")
print(f"  Selected augmentations : {result.selected_augmentations}")
print(f"  Report                 : {result.report_json_path}")
print("=" * 60)

# Show which augmentations were selected
if result.selected_augmentations:
    print("\nBNNR selected these augmentations (in order of stacking):")
    for i, name in enumerate(result.selected_augmentations, 1):
        tag = ""
        if name in ("icd", "aicd"):
            tag = " <- XAI-driven!"
        print(f"  {i}. {name}{tag}")
    print("\nAll other candidates were tested but did not improve accuracy.")

## 10. Inspect XAI Artifacts

BNNR saves saliency maps as images. Each image has three panels: **Original | Saliency Map | Overlay**.

In [ ]:
from pathlib import Path
from IPython.display import display, Image as IPImage
import glob

run_dir = result.report_json_path.parent
xai_files = sorted(glob.glob(str(run_dir / "artifacts" / "xai" / "**" / "*.png"), recursive=True))

if xai_files:
    print(f"Found {len(xai_files)} XAI images. Showing first 6:")
    for f in xai_files[:6]:
        print(f"\n{Path(f).name}:")
        display(IPImage(filename=f, width=600))
else:
    print("No XAI images found.")

# Also show augmentation previews if they exist
preview_files = sorted(glob.glob(str(run_dir / "artifacts" / "candidate_previews" / "*.png")))
if not preview_files:
    preview_files = sorted(glob.glob(str(run_dir / "artifacts" / "augmentation_previews" / "*.png")))
if preview_files:
    print(f"\nFound {len(preview_files)} augmentation preview images. Showing first 3:")
    for f in preview_files[:3]:
        print(f"\n{Path(f).name}:")
        display(IPImage(filename=f, width=600))

## 11. Explore the Events Log

Every training event is logged to `events.jsonl`. You can replay them to reconstruct the full training history.

In [ ]:
from bnnr import replay_events

events_path = run_dir / "events.jsonl"
if events_path.exists():
    events = replay_events(events_path)
    event_types = {}
    for e in events:
        t = e.get("event", "unknown")
        event_types[t] = event_types.get(t, 0) + 1

    print(f"Total events: {len(events)}")
    print("\nEvent types:")
    for t, count in sorted(event_types.items()):
        print(f"  {t:<35s} {count}")
else:
    print("No events.jsonl found.")

---

## 12. Dashboard (Post-Training)

If you're running locally, you can always replay the dashboard after training:

```bash
bnnr dashboard serve --run-dir reports/stl10_demo
```

The dashboard shows:
- **Branch tree**: Which augmentation was selected at each decision round
- **Metrics timeline**: Accuracy, F1, loss over epochs
- **Per-class metrics**: Accuracy per class (useful for imbalanced data)
- **Augmentation previews**: 512×512 side-by-side comparisons
- **XAI heatmaps**: Where the model focuses attention
- **Radar chart**: Comparing candidates

---

## Next Steps

- **Full STL-10**: Remove the `Subset(...)` lines and set `m_epochs=15`, `max_iterations=10`
- **Your own data**: Use `datasets.ImageFolder('path/to/data')` — see the [Custom Data Notebook](../bnnr_custom_data.ipynb)
- **Detection**: See the [Detection Demo](../detection/bnnr_detection_demo.ipynb)
- **Augmentations deep-dive**: See the [Augmentations Guide](../bnnr_augmentations_guide.ipynb)
- **ICD/AICD showcase**: See `examples/classification/showcase_stl10.py` for a full 100-epoch run with 20 candidates